# Twitter

In [ ]:
!pip install tweepy

In [ ]:
import tweepy

Create a new app by going to https://developer.twitter.com/en/portal/projects/1359950786231324672/apps/new

## Authentication

In [ ]:
consumer_key = ADD HERE 
consumer_secret = ADD HERE 
access_token = ADD HERE 
access_token_secret = ADD HERE 


(back to slides)
## Tweepy

In [ ]:
import tweepy
tweepy.__version__

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth

(back to slides)
## Tweepy Stream API

Access tweets in real time. We need to complete the following steps:

1. Create a class inheriting from StreamListener
1. Using that class create a Stream object
1. Connect to the Twitter API using the Stream.

#### Create a class that inherits from StreamListener

In [ ]:
class StreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        print(status.text)
        last_status = status
        
    def on_error(self, status_code):
        print('Got an error with status code: ' + str(status_code))

        if status_code == 420:
            return False

    def on_timeout(self):
        print('Timeout...')
        return True # To continue listening
    
StreamListener

#### Create a new Stream object

In [ ]:
stream_listener = StreamListener()
stream_listener

#### Authenticate the object

In [ ]:
stream = tweepy.Stream(auth=auth, listener=stream_listener)
stream

In [ ]:
stream.running

In [ ]:
stream

### Connet the Stream Object to the API

#### Sample Stream

In [ ]:
stream.sample()

In [ ]:
stream.disconnect()
#stream.sample(languages=['en'])
stream.running()

**Specify only Tweets in English**

In [ ]:
stream.sample(languages=['en'])

In [ ]:
stream.running

In [ ]:
stream.disconnect()
stream.running

#### Filter the Stream

Last night the Miami Heat were playing so I was filtering tweets about the team

In [ ]:
stream.disconnect()
stream.filter(track=["heat"] ,languages=["en"]) #is_async=True)

In [ ]:
status = stream.disconnect()
status, stream.running

Let's filtering on something trending right now

In [ ]:
stream.filter(track=[ADD HERE] ,languages=["en"]) 

In [ ]:
status

In [ ]:
stream.chunk_size

In [ ]:
stream.body

In [ ]:
stream.host

In [ ]:
stream.disconnect()
stream.running

In [ ]:
stream.chunk_size

In [ ]:
status

### Save Stream to File

In [ ]:
class StreamListener2(tweepy.StreamListener):
    
    def on_status(self, status):
        print(status.text)
        
    def on_data(self, data):
        #print data
        with open('fetched_tweets.txt','a') as tf:
            tf.write(data)
        return True
        
    def on_error(self, status_code):
        print('Got an error with status code: ' + str(status_code))

        if status_code == 420:
            return False

    def on_timeout(self):
        print('Timeout...')
        return True # To continue listening

stream_listener = StreamListener2()
stream2file = tweepy.Stream(auth=auth, listener=stream_listener)
stream2file

Let's keep this running for the remaining of class till we get to "analyze tweets" at the bottom

Let's look for some terms of what is currently trending on Twitter

https://twitter.com/explore/tabs/trending

**Asyncronous Streaming**

In [ ]:
stream2file.disconnect()
stream2file.filter(track=[ADD], is_async=True)

## Tweepy Rest API: 
https://docs.tweepy.org/en/latest/api.html#tweepy.API

In [ ]:
api = tweepy.API(auth)
api

### Search

Let's just run the following to get the 10 most recent Tweets that mention Barnard College

In [ ]:
top_ten_tweets = tweepy.Cursor(api.search, q='Barnard College').items(10)
for tweet in top_ten_tweets:
    print(tweet.text)

In [ ]:
[tweet.text for tweet in top_ten_tweets]

In [ ]:
for tweet in top_ten_tweets:
    print("barnard college" in tweet.text)

#### Elements from a Tweet

In [ ]:
" ".join(dir(tweet))

In [ ]:
tweet.text

In [ ]:
tweet.id

In [ ]:
tweet.created_at

In [ ]:
tweet.lang

In [ ]:
tweet.metadata

In [ ]:
tweet.in_reply_to_status_id

In [ ]:
tweet.source

In [ ]:
tweet.place

In [ ]:
tweet.coordinates

In [ ]:
tweet.entities

In [ ]:
tweet.retweeted

In [ ]:
tweet.retweet_count

In [ ]:
tweet.retweets()

To view the tweet online, we use the following syntax:
> twitter.com/*\<Screen Name\>*/status/*\<tweet id\>*

In [ ]:
tweet.user.screen_name, tweet.id

In [ ]:
f"twitter.com/{tweet.user.screen_name}/status/{tweet.id}"

List of all Attributes: 
https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

#### Author/User

In [ ]:
tweet.user == tweet.author

In [ ]:
author = tweet.author
author

In [ ]:
" ".join(dir(author))

In [ ]:
author.screen_name

In [ ]:
author.url

In [ ]:
author.id

**Question:** Who follows this person?

In [ ]:
" ".join([str(author_id) for author_id in author.followers_ids()][:10])

In [ ]:
author.followers()

#### Advanced Searche/Queries

https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators

##### No retweets

-filter:retweets

In [ ]:
ten_no_retweets = tweepy.Cursor(api.search, 
                                q='Barnard College -filter:retweets', 
                                count=10)

In [ ]:
items = ten_no_retweets.items(10)
items

In [ ]:
for tweet in items:
    print(tweet.text)

In [ ]:
type(item)

In [ ]:
all_tweets = []
for tweet in ten_no_retweets:
    all_tweets.append(tweet)

In [ ]:
tweet = all_tweets[0]


##### Safe Tweets

Tweets marked as potentially sensitive removed.

-filter:safe

In [ ]:
safe_cursor = tweepy.Cursor(api.search, 
                                q='Barnard College -filter:safe', 
                                count=10)

safe_tweets = safe_cursor.iterator.next()

In [ ]:
safe_tweets[0].text

### Users

In [ ]:
user = api.get_user(author.id)
user

In [ ]:
user_timeline = user.timeline()
user_timeline

In [ ]:
len(user_timeline)

In [ ]:
status_cursor = tweepy.Cursor(api.user_timeline, screen_name="billybob", count=200,tweet_mode='extended')
status_list = status_cursor.iterator.next()

In [ ]:
len(status_list)

In [ ]:
status_cursor = tweepy.Cursor(api.user_timeline, screen_name="potus", count=200,tweet_mode='extended')
status_list = status_cursor.iterator.next()

In [ ]:
len(status_list)

### Trends in a certain location

In [ ]:
nyc_trends = api.trends_place(id=2459115)
len(nyc_trends[0])

In [ ]:
nyc_trends[0].keys()

In [ ]:
len(nyc_trends[0]['trends'])

In [ ]:
nyc_trends[0]['trends'][6]

## Tweets from more than 7 days ago

>  Note that the standard search API only serves data from the last week. If you need historical data older than seven days, check out the premium and enterprise search APIs.

So we will often rely on a dataset of existing IDs.

**Terms and Conditions**

>The best place to get Twitter Content is directly from Twitter. Consequently, we restrict the redistribution of Twitter Content to third parties.  If you provide Twitter Content to third parties, including downloadable datasets or via an API, you may only distribute Tweet IDs, Direct Message IDs, and/or User IDs (except as described below). We also grant special permissions to academic researchers sharing Tweet IDs and User IDs for non-commercial research purposes.

> In total, you may not distribute more than 1,500,000 Tweet IDs to any entity (inclusive of multiple individuals associated with a single entity) within any 30 day period unless you have received written permission from Twitter. In addition, all developers may provide up to 50,000 public Tweets Objects and/or User Objects to each person who uses your service on a daily basis if this is done via non-automated means (e.g., download of spreadsheets or PDFs).

> Academic researchers are permitted to distribute an unlimited number of Tweet IDs and/or User IDs if they are doing so on behalf of an academic institution and for the sole purpose of non-commercial research. For example, you are permitted to share an unlimited number of Tweet IDs for the purpose of enabling peer review or validation of your research. If you have questions about whether your use case qualifies under this category please submit a request via the API Policy Support form.

**TLDR:** Don't share tweets, only TweetIDs

https://developer.twitter.com/en/developer-terms/agreement-and-policy

Here is an example of Tweet Ids of Tweets that are related to COVID-19: https://github.com/echen102/COVID-19-TweetIDs

### Finding Tweets by ID

`api.get_status()` ([documentation](https://docs.tweepy.org/en/latest/api.html?highlight=get_status#tweepy.API.get_status)) allows us to get a tweet by the tweet's id

In [ ]:
tweet = api.get_status(1256013432622481408)
tweet

In [ ]:
tweet.created_at

In [ ]:
f"twitter.com/{tweet.user.screen_name}/status/{tweet.id}"

## Checking rate limit status

In [ ]:
status_dict = api.rate_limit_status()
status_dict.keys()

## Academic Access
Twitter recently created an academic track for accessing the API that grants access to the entire API:
https://developer.twitter.com/en/solutions/academic-research/products-for-researchers. It grants academics access to Twitter data going back to 2006

Twitter [blog post](https://blog.twitter.com/developer/en_us/topics/tools/2021/enabling-the-future-of-academic-research-with-the-twitter-api.html)

## Analyze Tweets

In [ ]:
!wc fetched_tweets.txt

In [ ]:
[item for item in dir() if not item.startswith("_")]

In [ ]:
stream2file.disconnect()
stream2file.running

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("fetched_tweets.txt", lines=True)
df

In [ ]:
df.keys()

In [ ]:
df.shape

In [ ]:
df = df[df['retweeted_status'].isna()]
df.shape

In [ ]:
df['text'].shape

### Tokenize Tweets

In [ ]:
df['text'].iloc[1]

In [ ]:
nltk.word_tokenize(df['text'].iloc[1])

In [ ]:
twitter_tokenizer = nltk.tokenize.TweetTokenizer()
twitter_tokenizer.tokenize(df['text'].iloc[1])

In [ ]:
twitter_tokenizer.strip_handles = True
twitter_tokenizer.tokenize(df['text'].iloc[1])

In [ ]:
import nltk


In [ ]:
nltk.FreqDist(nltk.word_tokenize(" ".join(df['text']))

In [ ]:
nltk.FreqDist(" ".join([" ".join(sent) for sent in df['text'].str.lower().apply(nltk.word_tokenize)]).split())

In [ ]:
df['text_tokenize'] = df['text'].apply(twitter_tokenizer)
df['text_tokenize']